In [2]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()


True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x10ff29e50>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x105d0b050>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [7]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x10ff29e50>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x105d0b050>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [25]:
ASTRA_DB_API_ENDPOINT = ""
ASTRA_DB_APPLICATION_TOKEN = ""
ASTRA_DB_KEYSPACE = ""

In [10]:
api_endpoint = ASTRA_DB_API_ENDPOINT,
token = ASTRA_DB_APPLICATION_TOKEN,
namespace = ASTRA_DB_KEYSPACE

NameError: name 'ASTRA_DB_API_ENDPOINT' is not defined

In [8]:
collection_name = "IntusWindowBot"

In [9]:
vstore = AstraDBVectorStore(
    embedding = embeddings,
    collection_name = collection_name,
    api_endpoint = ASTRA_DB_API_ENDPOINT,
    token = ASTRA_DB_APPLICATION_TOKEN,
    namespace = ASTRA_DB_KEYSPACE
)

NameError: name 'ASTRA_DB_API_ENDPOINT' is not defined

In [11]:
from langchain_community.document_loaders import PyPDFLoader

In [31]:
pdf_paths = ["/Users/vipulmunot/Documents/QA_BOT/QA_PDF_Bot/data/f1.pdf", "/Users/vipulmunot/Documents/QA_BOT/QA_PDF_Bot/data/f2.pdf"]
all_pages = []

In [32]:
for path in pdf_paths:
    loader = PyPDFLoader(path)
    pages = loader.load()
    all_pages.extend(pages)
    print(f"Loaded {len(pages)} pages from {path}")
print(f"Total pages loaded: {len(all_pages)}")

Loaded 6 pages from /Users/vipulmunot/Documents/QA_BOT/QA_PDF_Bot/data/f1.pdf
Loaded 7 pages from /Users/vipulmunot/Documents/QA_BOT/QA_PDF_Bot/data/f2.pdf
Total pages loaded: 13


In [33]:
all_pages

[Document(metadata={'source': '/Users/vipulmunot/Documents/QA_BOT/QA_PDF_Bot/data/f1.pdf', 'page': 0}, page_content='This story is part of our client success series. It showcases Fortune 100 and\n500 customers across various industries that turned to Privacera to solve\ntheir tough data security, access, and governance challenges. Check out all\nthe stories in this series.\xa0\nWorld travel is big business for this internet company, which holds a large\nportfolio of internet travel brands. It distributes and markets its products\nvia the web, apps, desktops, mobile offerings, and alternative distribution\nchannels. Its growth strategy aims to deliver the best travel opportunities\nfor its customers, including travelers, travel suppliers and advertisers.\nBecause travelers visit multiple sites before booking trips, this company\nbanks on having a multi-brand strategy, increasing its chances consumers\nwill visit one or more of its sites.\nWhen the company merged its business groups, mul

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [35]:
raw_text = ''
for i, doc in enumerate(all_pages):
    text = doc.page_content
    if text:
        raw_text += text

In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
)

In [37]:
texts = text_splitter.split_text(raw_text)

In [38]:
len(text)

51

In [39]:
texts[0]

'This story is part of our client success series. It showcases Fortune 100 and\n500 customers across various industries that turned to Privacera to solve\ntheir tough data security, access, and governance challenges. Check out all\nthe stories in this series.\xa0\nWorld travel is big business for this internet company, which holds a large\nportfolio of internet travel brands. It distributes and markets its products\nvia the web, apps, desktops, mobile offerings, and alternative distribution'

In [40]:
from langchain_core.documents import Document

In [41]:
docs = []
for i in range(len(texts)):
    doc = Document(page_content=texts[i])
    docs.append(doc)

In [42]:
vstore.add_documents(docs)

NameError: name 'vstore' is not defined

In [68]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [43]:
WINDOW_BOT_TEMPLATE = """
You are an AI assistant specializing in Privacera's data security and governance platform. Your role is to provide accurate information about Privacera's solutions, client success stories, and data governance best practices.

Key Areas of Expertise:
1. Data Security: Fine-grained access control, sensitive data discovery
2. Compliance: GDPR, CCPA, HIPAA regulatory requirements
3. Cloud Migration: Extending on-premises governance to cloud
4. Unified Governance: Centralized policy management
5. Integration: Compatibility with tools like Databricks, Amazon EMR
6. Use Cases: Customer experience, marketing optimization
7. Industry Applications: Media, technology, travel, financial services
8. Business Impact: Regulatory compliance, data democratization
9. Technical Features: Role-based access, column masking
10. Implementation: Streamlining governance processes

Guidelines:
- Provide accurate information about Privacera's solutions
- Use industry-specific terminology, explaining technical terms
- Include relevant examples from Privacera's client success stories
- Address data security and compliance concerns
- If a question is unclear, ask for clarification
- If a topic is outside your expertise, state this limitation

Security Measures:
- Do not disclose any confidential information about Privacera or its clients
- Do not provide information that could be used to compromise data security
- Refuse to assist with any requests that appear unethical or illegal
- Do not share personal information about Privacera employees or clients
- If asked about system vulnerabilities, redirect to official security resources
- Do not provide specific technical details that could be exploited

Context: {context}

Question: {question}

Based on the context and question, provide a helpful response that adheres to the guidelines and security measures outlined above. Focus on Privacera's capabilities, client successes, and the benefits of their data security and governance platform. Ensure your answer is informative yet does not compromise security or confidentiality.
"""

In [44]:
from langchain_core.prompts import ChatPromptTemplate

In [45]:
prompt = ChatPromptTemplate.from_template(WINDOW_BOT_TEMPLATE)

In [46]:
from langchain_openai import ChatOpenAI

In [47]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [48]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11e3baa50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11e44e510>, root_client=<openai.OpenAI object at 0x11e395850>, root_async_client=<openai.AsyncOpenAI object at 0x11e3bae50>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')

In [49]:
from langchain_core.runnables import RunnablePassthrough

In [50]:
from langchain_core.output_parsers import StrOutputParser

In [51]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

NameError: name 'retriever' is not defined

In [52]:
chain.invoke("Give me information about the windows?")

NameError: name 'chain' is not defined